In [1]:
import pandas as pd
pd.__version__

'1.3.0.dev0+1235.g77ea98e9b7'

In [23]:
quotes = pd.DataFrame({
    "time": [
        pd.Timestamp("2016-05-25 13:30:00.023"),
        pd.Timestamp("2016-05-25 13:30:00.023"),
        pd.Timestamp("2016-05-25 13:30:00.030"),
        pd.Timestamp("2016-05-25 13:30:00.041"),
        pd.Timestamp("2016-05-25 13:30:00.048"),
        pd.Timestamp("2016-05-25 13:30:00.049"),
        pd.Timestamp("2016-05-25 13:30:00.072"),
        pd.Timestamp("2016-05-25 13:30:00.075")
        ],
    "ticker": [
        "GOOG",
        "MSFT",
        "MSFT",
        "MSFT",
        "GOOG",
        "AAPL",
        "GOOG",
        "MSFT"
        ],
    "bid": [720.50, 51.95, 51.97, 51.99, 720.50, 97.99, 720.50, 52.01],
    "ask": [720.93, 51.96, 51.98, 52.00, 720.93, 98.01, 720.88, 52.03]
    })
quotes

,time,ticker,bid,ask
0,2016-05-25 13:30:00.023,GOOG,720.50,720.93
1,2016-05-25 13:30:00.023,MSFT,51.95,51.96
2,2016-05-25 13:30:00.030,MSFT,51.97,51.98
3,2016-05-25 13:30:00.041,MSFT,51.99,52.00
4,2016-05-25 13:30:00.048,GOOG,720.50,720.93
5,2016-05-25 13:30:00.049,AAPL,97.99,98.01
6,2016-05-25 13:30:00.072,GOOG,720.50,720.88
7,2016-05-25 13:30:00.075,MSFT,52.01,52.03


In [24]:
trades = pd.DataFrame({
    "time": [
        pd.Timestamp("2016-05-25 13:30:00.023"),
        pd.Timestamp("2016-05-25 13:30:00.038"),
        pd.Timestamp("2016-05-25 13:30:00.048"),
        pd.Timestamp("2016-05-25 13:30:00.048"),
        pd.Timestamp("2016-05-25 13:30:00.048")
        ],
    "ticker": ["MSFT", "MSFT", "GOOG", "GOOG", "AAPL"],
    "kind": ["BBG", "ARCA", "BBG", "BBG", "ARCA"],
    "price": [51.95, 51.95, 720.77, 720.92, 98.0],
    "quantity": [75, 155, 100, 100, 100]
    })
trades

,time,ticker,kind,price,quantity
0,2016-05-25 13:30:00.023,MSFT,BBG,51.95,75
1,2016-05-25 13:30:00.038,MSFT,ARCA,51.95,155
2,2016-05-25 13:30:00.048,GOOG,BBG,720.77,100
3,2016-05-25 13:30:00.048,GOOG,BBG,720.92,100
4,2016-05-25 13:30:00.048,AAPL,ARCA,98.00,100


In [27]:
# typical merge_asof
pd.merge_asof(quotes, trades, on='time', by='ticker')

,time,ticker,bid,ask,kind,price,quantity
0,2016-05-25 13:30:00.023,GOOG,720.50,720.93,NaN,NaN,NaN
1,2016-05-25 13:30:00.023,MSFT,51.95,51.96,BBG,51.95,75.0
2,2016-05-25 13:30:00.030,MSFT,51.97,51.98,BBG,51.95,75.0
3,2016-05-25 13:30:00.041,MSFT,51.99,52.00,ARCA,51.95,155.0
4,2016-05-25 13:30:00.048,GOOG,720.50,720.93,BBG,720.92,100.0
5,2016-05-25 13:30:00.049,AAPL,97.99,98.01,ARCA,98.00,100.0
6,2016-05-25 13:30:00.072,GOOG,720.50,720.88,BBG,720.92,100.0
7,2016-05-25 13:30:00.075,MSFT,52.01,52.03,ARCA,51.95,155.0


In [33]:
# multi-by merge_asof, note that the kind *only* appears on the rhs
pd.merge_asof(quotes, trades, on='time', left_by='ticker', right_by=['ticker', 'kind'])

MergeError: left_by and right_by must be same length

In [35]:
# simulate the above
left_by = {t:df for t, df in quotes.groupby('ticker')}

In [39]:
result = pd.concat([pd.merge_asof(left_by[ticker], frame, on='time', by='ticker')
                   for (ticker, kind), frame in trades.groupby(['ticker', 'kind'])
                   if left_by.get(ticker) is not None], ignore_index=True)
result

,time,ticker,bid,ask,kind,price,quantity
0,2016-05-25 13:30:00.049,AAPL,97.99,98.01,ARCA,98.00,100.0
1,2016-05-25 13:30:00.023,GOOG,720.50,720.93,NaN,NaN,NaN
2,2016-05-25 13:30:00.048,GOOG,720.50,720.93,BBG,720.92,100.0
3,2016-05-25 13:30:00.072,GOOG,720.50,720.88,BBG,720.92,100.0
4,2016-05-25 13:30:00.023,MSFT,51.95,51.96,NaN,NaN,NaN
5,2016-05-25 13:30:00.030,MSFT,51.97,51.98,NaN,NaN,NaN
6,2016-05-25 13:30:00.041,MSFT,51.99,52.00,ARCA,51.95,155.0
7,2016-05-25 13:30:00.075,MSFT,52.01,52.03,ARCA,51.95,155.0
8,2016-05-25 13:30:00.023,MSFT,51.95,51.96,BBG,51.95,75.0
9,2016-05-25 13:30:00.030,MSFT,51.97,51.98,BBG,51.95,75.0
